In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import read_quote      as rq
import stock_functions as sf
import df_visualizations as dv
import remap_values as rv

import random
import pickle

import sys
import time

In [2]:
from sklearn.svm            import SVR
from sklearn.ensemble       import RandomForestRegressor
from sklearn.linear_model   import ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble       import BaggingRegressor
from sklearn.ensemble       import AdaBoostRegressor

from sklearn.utils          import shuffle

from sklearn.metrics        import mean_squared_error

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.multioutput    import MultiOutputRegressor

In [12]:
roll_nums = [5,15]#[3,5,7,10,12,15,18,20,22,25,28,30] # Generate an algorithm for predicting every few days
mom_nums  = [3,5,8,10,15,20,25,30]                    # Momentum has many good tracers
rsi_nums  = [10,15,20,25,30]                          # Good for some long term trends
band_nums = [5,8,10,15,20,25]                         # A few trace different areas well

In [4]:
inpFileList = ['aapl','acm' ,'awk' ,'amzn','awr' ,'ba'  ,'bac' ,'c'   ,'cat' ,'cop' ,'cvx' ,'dal' ,
               'dd'  ,'farm','fdp' ,'gnc' ,'hes' ,'ibm' ,'mas' ,'mcd' ,'mon' ,'msex','msft','nflx',
               'sbux','strl','tgt' ,'tsla','ups' ,'xom' ,'xpo' ,'vmc' ]

In [9]:
reload(sf)

<module 'stock_functions' from 'stock_functions.py'>

In [13]:
scaled_df = sf.get_scaled_data( inpFileList, roll_nums, mom_nums, rsi_nums, band_nums )

In [17]:
scaled_df = sf.gen_pca_attributes( scaled_df, mom_nums, rsi_nums, band_nums, n_mom=3, n_rsi=2, n_ban=2 )

In [9]:
# Read in each stock, and perform reduction
#    Break into train, test sets
#    Store train, test sets in arrays
# For each regressor predicting a certain num of days...
#    Manually cross validate, testing and scoring each set

In [21]:
# Mix up the list
shuff_scaled_df = shuffle( scaled_df, random_state=0 )


# Seperate features from target data
close_list = ['close']
for i in roll_nums:
    close_list.append( 'close_mean_'+str(i) )

targets  = shuff_scaled_df[close_list]
features = shuff_scaled_df.drop( close_list, axis=1 )



n_elements = shuff_scaled_df.shape[0]

# Set the fraction of data we will train over
percent_train = 0.7

# Size of training set
n_train = int( percent_train * n_elements )


# Seperate training and test data
train_features = features[ : n_train   ]
test_features  = features[   n_train : ]

train_targets  =  targets[ : n_train   ]
test_targets   =  targets[   n_train : ]

In [33]:
reg_list = []

# Features are the same, but targets training on will vary
# As will which features missing ( due to ways rolling mean calculated )
# Skip the next days, close, but otherwise use rolling means
for col in close_list[1:]:

    # Gets the location of null values
    train_not_null_locations = train_targets[col].isnull()==0
    test_not_null_locations  =  test_targets[col].isnull()==0
    
    # Take features and targets that are not null
    train_x = train_features[ train_not_null_locations ].values
    test_x  =  test_features[  test_not_null_locations ].values
    
    train_y = train_targets[ train_not_null_locations ][col].values
    test_y  =  test_targets[  test_not_null_locations ][col].values
    
    reg_list.append( AdaBoostRegressor( base_estimator=RandomForestRegressor( n_estimators=20, max_features=0.7 ), random_state=0, 
                            loss='exponential', n_estimators=10 ) )

    reg_list[-1].fit( train_x, train_y )
    
    ad_y = reg_list[-1].predict( test_x )
    
    print col, ' MSE : ', mean_squared_error( ad_y, test_y )

close_mean_5  MSE :  0.00126550114664
close_mean_15  MSE :  0.00254971575641
